In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_squared_error

In [8]:
# 1. Importar el df del otro script
import limpieza
df = limpieza.df_transformado

Las columnas de alquiler y compra son las mismas? -->  True
Hay 3 registros duplicados en df. Se muestran en el df_duplicados
Se han eliminado los duplicados
Análisis de la columna bathrooms
Valores únicos:
['1', '2', '3', '4', '6']

Conteo de valores:
bathrooms
1    452
2    275
3     38
4      9
6      1
Name: count, dtype: int64

Resumen:
count    775.000000
mean       1.494194
std        0.666399
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        6.000000
Name: bathrooms, dtype: float64

Valores nulos: 0

--------------------------------------------------

Análisis de la columna exterior
Valores únicos:
['False', 'True', 'nan']

Conteo de valores:
exterior
True     692
False     51
Name: count, dtype: int64

Resumen:
count      743
unique       2
top       True
freq       692
Name: exterior, dtype: object

Valores nulos: 32

--------------------------------------------------

Análisis de la columna floor
Valores únicos:
['1', '10', '12', '13'

In [9]:
# 2. Separar la variable objetivo ('price') y las variables independientes
X = df.drop('price', axis=1)
y = df['price']

# 3. Dividir en conjuntos de entrenamiento y prueba (esto solo será utilizado para comparar luego)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# 4. Escalar las variables numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# 5. Crear el modelo de árbol de regresión
regressor = DecisionTreeRegressor(max_depth = 10, min_impurity_decrease = 0.01, min_samples_leaf = 2,splitter = 'random')

#{'criterion': 'squared_error', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'splitter': 'random'}


In [7]:
# GRID SEARCH
# Definir el espacio de hiperparámetros para la búsqueda
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'min_impurity_decrease': [0.0, 0.01, 0.1]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Ejecutar GridSearchCV
grid_search.fit(X_train, y_train)

# Mejor combinación de hiperparámetros encontrada
print("Mejores hiperparámetros:", grid_search.best_params_)

# Mejor modelo entrenado
best_model = grid_search.best_estimator_

Fitting 10 folds for each of 15552 candidates, totalling 155520 fits


C:\Users\Javi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
38880 fits failed out of a total of 155520.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12623 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Javi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Javi\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\Javi\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Javi\anaconda3\Lib\site-pack

Mejores hiperparámetros: {'criterion': 'squared_error', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'splitter': 'random'}


C:\Users\Javi\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [-131168.68807509 -128573.10754892 -157764.49245612 ... -240001.0414127
 -215151.80360949 -239530.23695178]
  warnings.warn(


In [29]:
# Predecir sobre el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error en test set:", mse)

Mean Squared Error en test set: 105019.37330739645


In [30]:
# 6. Validación cruzada
# Nota: Si estás usando StandardScaler, deberías escalar dentro de la validación cruzada (ej: con Pipelines o ColumnTransformers)
rmse_scorer = make_scorer(mean_squared_error, squared=False)  # Definir el RMSE como métrica
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=10, scoring=rmse_scorer)

# Mostrar resultados de la validación cruzada
print(f"RMSE scores for each fold: {cv_scores}")
print(f"Mean RMSE: {cv_scores.mean():.2f}")
print(f"Standard Deviation of RMSE: {cv_scores.std():.2f}")

RMSE scores for each fold: [337.13342184 336.92661558 263.00766631 325.32921121 297.63670278
 344.91974735 246.23719898 331.31746763 273.19010805 225.32431856]
Mean RMSE: 298.10
Standard Deviation of RMSE: 41.12


In [31]:
# 7. Entrenar el modelo en todos los datos de entrenamiento
best_model.fit(X_train_scaled, y_train)

DecisionTreeRegressor(criterion='friedman_mse', max_depth=50,
                      min_samples_leaf=4, min_samples_split=10,
                      splitter='random')

In [32]:
# 8. Hacer predicciones y evaluar el modelo
y_pred = best_model.predict(X_test_scaled)
df_resultados = pd.DataFrame({'Real': y_test, 
                              'Predicted': y_pred, 
                              'PorcentajeError': abs((y_pred - y_test) / y_test) * 100})
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Final Root Mean Squared Error on Test Set: {rmse:.2f}")

Final Root Mean Squared Error on Test Set: 307.63


In [10]:
import metricas

In [13]:
y_pred = regressor.predict(X_test)


NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
metricas()